In [1]:
from pymongo import MongoClient
import yfinance as yf
from datetime import datetime

In [2]:
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')  # Adjust connection string as needed
db = client['financial_data']

# Create collections
historical_collection = db['historical_stock_data']
realtime_collection = db['realtime_stock_data']
news_collection = db['financial_news']

In [3]:
# Define the stock ticker symbol
ticker_symbol = "AAPL"

# Collect historical stock data
stock_data = yf.download(ticker_symbol, start="2023-01-01", end="2023-12-31")
historical_data_list = stock_data.reset_index().to_dict(orient='records')

# Collect real-time stock data
ticker = yf.Ticker(ticker_symbol)
realtime_data = ticker.history(period="1d").reset_index().to_dict(orient='records')

# Collect financial news
news = ticker.news
news_data_list = []
for article in news:
    news_data_list.append({
        "title": article['title'],
        "date": datetime.fromtimestamp(article['providerPublishTime']),
        "link": article['link'],
        "source": article['publisher']
    })

[*********************100%%**********************]  1 of 1 completed


In [4]:
# Insert historical stock data
if historical_data_list:
    historical_collection.insert_many(historical_data_list)
    print(f"Inserted {len(historical_data_list)} historical records.")

# Insert real-time stock data
if realtime_data:
    realtime_collection.insert_many(realtime_data)
    print(f"Inserted {len(realtime_data)} real-time records.")

# Insert financial news data
if news_data_list:
    news_collection.insert_many(news_data_list)
    print(f"Inserted {len(news_data_list)} news articles.")

Inserted 250 historical records.
Inserted 1 real-time records.
Inserted 8 news articles.
